## Bidirectional Neural Representation:

In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.compat.v1.disable_eager_execution()

2024-09-03 00:04:19.631206: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 00:04:20.683071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##### Bipolar permutation function $f: \{-1,1\}^n \rightarrow \{-1, 1\}^n$
##### Binary permutation function $f : \{0,1\}^n \rightarrow \{0, 1\}^n$

In [2]:
# 3-bit and 4-bit bipolar permutation
def get_data(data_type = "3_bits"):
    """
    Generate the training dataset for a pre-defined 
    3-bit or 4-bit permutation function
    """
    
    if data_type == "3_bits":
        x = np.array([
            [1,1,1], [1,1,-1], [1,-1,1], [1,-1,-1],
            [-1,1,1], [-1,1,-1], [-1,-1,1], [-1,-1,-1]])
        y = x[[6,4,2,0,7,5,3,1]]      
    elif data_type == "4_bits":
        x = np.array([
            [-1,-1,-1,-1],[-1,-1,-1,1],[-1,-1,1,-1],[-1,-1,1,1],
            [-1,1,-1,-1],[-1,1,-1,1],[-1,1,1,-1],[-1,1,1,1],
            [1,-1,-1,-1],[1,-1,-1,1],[1,-1,1,-1],[1,-1,1,1],
            [1,1,-1,-1],[1,1,-1,1],[1,1,1,-1],[1,1,1,1]])
        y = x[[12, 14, 15, 13, 9, 5, 8, 4, 11, 7, 10, 6, 2, 0, 1, 3]]
    else:
        raise ValueError("Wrong input!!! Data type should be 3_bits or 4_bits")
    return x, y

In [3]:
data_type = "3_bits"  # "4_bit"
num_hidden_neurons = 4
mode = 'binary'  # "bipolar"

In [4]:
X, Y = get_data(data_type)
if mode == 'binary':
  X = 0.5 * (X+1)
  Y = 0.5 * (Y+1)

###### Network Architecture: Input layer <--> Hidden Layer <--> Output Layer

In [7]:
## Define the Placeholders !!!
x_f = tf.compat.v1.placeholder(tf.float32, [None, X.shape[1]], name = 'input_forward')
y_f = tf.compat.v1.placeholder(tf.float32, [None, Y.shape[1]], name = 'target_forward')

x_b = tf.compat.v1.placeholder(tf.float32, [None, X.shape[1]], name = 'input_backward')
y_b = tf.compat.v1.placeholder(tf.float32, [None, Y.shape[1]], name = 'target_backward')

In [9]:
## Initialize weight
wgts = {}
wgts['U'] = tf.Variable(
    tf.compat.v1.random_normal(
        [X.shape[1], num_hidden_neurons], 
        stddev = 1.0)
    )
wgts['W'] = tf.Variable(
    tf.compat.v1.random_normal(
        [Y.shape[1], num_hidden_neurons], 
        stddev = 1.0)
    )